1dcnn 실시 (실험)

In [2]:
# pip install biosppy nolds scikit-learn peakutils

In [1]:
import wfdb
import numpy as np
import pandas as pd
import glob
import os
from wfdb import rdrecord
import numpy as np
import pandas as pd
import scipy.signal as signal
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
from scipy.stats import entropy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from scipy import signal, fft
from scipy.signal import spectrogram
from biosppy.signals.ecg import christov_segmenter, hamilton_segmenter
from nolds import sampen
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd
import pywt
import wfdb
from wfdb.processing import gqrs_detect
from scipy.signal import butter, filtfilt, welch, find_peaks
from scipy.stats import entropy
from multiprocessing import Pool, cpu_count
from sklearn.impute import SimpleImputer
import time
import scipy.ndimage
from scipy.stats import skew, kurtosis
from tqdm import tqdm
from math import atan2, degrees

2025-03-28 07:56:54.690828: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-28 07:56:54.692029: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-28 07:56:54.695811: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-28 07:56:54.705561: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743148614.721576     763 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743148614.72

### 모델 아키텍쳐 정의

In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tqdm import tqdm

import numpy as np
from scipy.signal import butter, filtfilt
from tqdm import tqdm
import pywt
import scipy.ndimage

# --- 전처리 함수들 ---
def wavelet_denoise(signal, wavelet='db6', level=4):
    try:
        coeffs = pywt.wavedec(signal, wavelet, level=level)
        threshold = np.median(np.abs(coeffs[-1])) / 0.6745
        coeffs[1:] = [pywt.threshold(c, threshold, mode='soft') for c in coeffs[1:]]
        return pywt.waverec(coeffs, wavelet)
    except:
        return signal

def oc_co_filter(signal, fs, k_factor):
    try:
        k_size = int(np.round(k_factor * fs))
        struct_elem = np.ones(k_size)
        opened = scipy.ndimage.grey_opening(signal, structure=struct_elem)
        closed = scipy.ndimage.grey_closing(opened, structure=struct_elem)
        return scipy.ndimage.grey_opening(closed, structure=struct_elem)
    except:
        return signal

def bandpass_filter(signal, fs=400, lowcut=0.1, highcut=45):
    try:
        nyq = 0.5 * fs
        b, a = butter(4, [lowcut / nyq, highcut / nyq], btype='band')
        return filtfilt(b, a, signal)
    except:
        return signal

# --- 최종 통합 전처리 + padding 함수 ---
def prepare_ecg_for_cnn(df, lead_index=0, max_len=3000, fs=400, target_col="Chagas_label"):
    """
    Wavelet + Baseline 제거 후 CNN 입력용 padding 적용
    """
    X = []
    y = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc="🔧 CNN용 ECG 전처리"):
        signal = row["p_signal"][:, lead_index]
        
        # ✅ 전처리: 밴드패스 → 웨이블릿 → 기저선 제거
        signal = bandpass_filter(signal, fs)
        signal = wavelet_denoise(signal)
        baseline = oc_co_filter(signal, fs, 0.27)
        signal = signal - baseline

        # ✅ padding or truncate
        if len(signal) >= max_len:
            signal = signal[:max_len]
        else:
            signal = np.pad(signal, (0, max_len - len(signal)), mode='constant')

        X.append(signal)
        y.append(row[target_col])

    X = np.array(X)[..., np.newaxis]  # (batch, time, 1)
    y = np.array(y)
    
    return X, y


# ✅ 모델 정의
def build_1d_cnn_model(input_shape):
    model = Sequential([
        Conv1D(32, kernel_size=5, activation='relu', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),

        Conv1D(64, kernel_size=5, activation='relu'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),

        Conv1D(128, kernel_size=5, activation='relu'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),

        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# ✅ 학습 및 평가 함수
def train_and_evaluate_1dcnn(df, lead_index=0, max_len=3000):
    X, y = prepare_ecg_for_cnn(df, lead_index=lead_index, max_len=max_len)
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

    model = build_1d_cnn_model(input_shape=X_train.shape[1:])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=10,
        batch_size=32,
        callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
        verbose=1
    )

    # 평가
    y_pred_prob = model.predict(X_test).flatten()
    y_pred = (y_pred_prob > 0.3).astype(int)
    
    print("✅ Accuracy:", accuracy_score(y_test, y_pred))
    print("✅ F1 Score:", f1_score(y_test, y_pred))
    print("\nClassification Report:\n", classification_report(y_test, y_pred))
    return model


### code15 데이터 처리

피처 추출 (lead1: index0)

In [2]:
df_part0 = pd.read_pickle("/workspace/my_data/code15_part0.pkl")
# df_part1 = pd.read_pickle("/workspace/my_data/code15_part1.pkl")

In [13]:
# 예: lead_index=0번 리드 사용, 길이 3000 고정
model = train_and_evaluate_1dcnn(df_part0, lead_index=0, max_len=3000)

/opt/conda/lib/python3.10/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 4 is too high: all coefficients will experience boundary effects.                    | 9155/19901 [00:38<00:45, 235.84it/s]
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/pywt/_thresholding.py:22: RuntimeWarning: invalid value encountered in divide
  thresholded = (1 - value/magnitude)
🔧 CNN용 ECG 전처리: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19901/19901 [01:23<00:00, 237.37it/s]
/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
498/498 ━━━━━━━━━━━━━━━━━━━━ 47s 91ms/step - accuracy: 0.9618 - loss: 0.6584 - val_accuracy: 0.9799 - val_loss: 0.4849
Epoch 2/10
498/498 ━━━━━━━━━━━━━━━━━━━━ 42s 84ms/step - accuracy: 0.9800 - loss: 0.4465 - val_accuracy: 0.9799 - val_loss: 0.3507
Epoch 3/10
498/498 ━━━━━━━━━━━━━━━━━━━━ 41s 82ms/step - accuracy: 0.9792 - loss: 0.3278 - val_accuracy: 0.9799 - val_loss: 0.2672
Epoch 4/10
498/498 ━━━━━━━━━━━━━━━━━━━━ 42s 83ms/step - accuracy: 0.9795 - loss: 0.2525 - val_accuracy: 0.9799 - val_loss: 0.2127
Epoch 5/10
498/498 ━━━━━━━━━━━━━━━━━━━━ 41s 83ms/step - accuracy: 0.9783 - loss: 0.2053 - val_accuracy: 0.9799 - val_loss: 0.1763
Epoch 6/10
498/498 ━━━━━━━━━━━━━━━━━━━━ 41s 83ms/step - accuracy: 0.9813 - loss: 0.1668 - val_accuracy: 0.9799 - val_loss: 0.1515
Epoch 7/10
498/498 ━━━━━━━━━━━━━━━━━━━━ 41s 83ms/step - accuracy: 0.9807 - loss: 0.1449 - val_accuracy: 0.9799 - val_loss: 0.1343
Epoch 8/10
498/498 ━━━━━━━━━━━━━━━━━━━━ 41s 83ms/step - accuracy: 0.9813 - loss: 0.1277 - 

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [10]:
from helper_code import compute_challenge_score  # 헬퍼 코드 불러오기

# ✅ 딥러닝 모델에서 확률값 얻기
y_pred_prob = model.predict(X_test).flatten()  # (batch_size, 1) → (batch_size,)

# ✅ Challenge Score 계산
challenge_score = compute_challenge_score(y_test, y_pred_prob)

print(f"Challenge Score: {challenge_score:.3f}")

NameError: name 'X_test' is not defined

In [39]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Add

# ✅ ResNet Block
def resnet_block(x, units, dropout_rate=0.3):
    shortcut = x  # Residual Connection

    x = Dense(units, activation="relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(units, activation="relu")(x)
    x = BatchNormalization()(x)
    
    # Residual Connection
    x = Add()([x, shortcut])
    return x

# ✅ ResNet 모델 생성
def build_resnet(input_dim):
    inputs = Input(shape=(input_dim,))
    
    x = Dense(128, activation="relu")(inputs)
    x = resnet_block(x, 128)
    x = resnet_block(x, 128)
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.3)(x)
    x = Dense(1, activation="sigmoid")(x)  
    
    model = Model(inputs, x)
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

# ✅ ResNet 모델 학습
resnet_model = build_resnet(X_train.shape[1])
resnet_model.fit(X_train, y_train, epochs=30, batch_size=64, verbose=1)

# ✅ 예측값 생성 (확률 값 사용)
resnet_train_preds = resnet_model.predict(X_train).flatten()
resnet_test_preds = resnet_model.predict(X_test).flatten()

Epoch 1/30


W0000 00:00:1743089047.904966      30 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


249/249 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8752 - loss: 0.3498
Epoch 2/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9798 - loss: 0.1156  
Epoch 3/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9785 - loss: 0.1206  
Epoch 4/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9793 - loss: 0.1163  
Epoch 5/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9796 - loss: 0.1077  
Epoch 6/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9810 - loss: 0.1054  
Epoch 7/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9814 - loss: 0.0997  
Epoch 8/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9802 - loss: 0.1055  
Epoch 9/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9770 - loss: 0.1179  
Epoch 10/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9777 - loss: 0.1144  
Epoch 11/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9778 - loss: 0.1142  
Epoch 12/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s

In [40]:
# ✅ ResNet 모델 Accuracy 평가 (Keras 내장 evaluate 함수 사용)
resnet_accuracy = resnet_model.evaluate(X_test, y_test, verbose=0)[1]  # [0]: loss, [1]: accuracy
resnet_preds_test = (resnet_model.predict(X_test) > 0.5).astype(int)
resnet_f1 = f1_score(y_test, resnet_preds_test)

print(f"ResNet Model Accuracy: {resnet_accuracy:.4f}")
print(f"ResNet Model F1 Score: {resnet_f1:.4f}")

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 934us/step
ResNet Model Accuracy: 0.9799
ResNet Model F1 Score: 0.0000


Stacking 해보기

In [42]:
from catboost import CatBoostClassifier

# ✅ CatBoost 모델 학습
catboost_model = CatBoostClassifier(n_estimators=1000, depth=6, learning_rate=0.05, verbose=0)
catboost_model.fit(X_train, y_train)

# ✅ 예측값 생성 (확률 값 사용)
catboost_train_preds = catboost_model.predict_proba(X_train)[:, 1]
catboost_test_preds = catboost_model.predict_proba(X_test)[:, 1]

In [43]:
import numpy as np

# Meta 모델을 위한 새로운 입력 데이터 생성
X_train_meta = np.column_stack((resnet_train_preds, catboost_train_preds))
X_test_meta = np.column_stack((resnet_test_preds, catboost_test_preds))

# Meta Model (CatBoost) 학습
meta_model = CatBoostClassifier(n_estimators=500, depth=6, learning_rate=0.05, verbose=0)
meta_model.fit(X_train_meta, y_train)

# Meta Model 예측
meta_preds_test = meta_model.predict(X_test_meta)


In [44]:
from sklearn.metrics import accuracy_score, f1_score

meta_acc = accuracy_score(y_test, meta_preds_test)
meta_f1 = f1_score(y_test, meta_preds_test)

print(f"✅ Meta Model Accuracy: {meta_acc:.4f}")
print(f"✅ Meta Model F1 Score: {meta_f1:.4f}")

# ✅ Meta Model Accuracy: 0.9754
# ✅ Meta Model F1 Score: 0.1091

✅ Meta Model Accuracy: 0.9754
✅ Meta Model F1 Score: 0.1091


In [45]:
# Challenge score

from helper_code import compute_challenge_score  # helper_code에서 함수 가져오기

# 모델 예측값 (확률 기반)
y_pred_prob = meta_model.predict_proba(X_test)[:, 1]  # 모델이 1(양성)일 확률 예측

# 실제 라벨
y_true = y_test  # 실제 정답 레이블 (0 또는 1)

# ✅ Challenge Score 계산
challenge_score = compute_challenge_score(y_true, y_pred_prob)

print(f"Challenge Score: {challenge_score:.3f}")

# Challenge Score: 0.159

Challenge Score: 0.000


RBBB 변수 추가